In [1]:
import os
import sys
import gensim
import pandas as pd
from gensim.models.doc2vec import LabeledSentence
from sklearn.cross_validation import train_test_split
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm
from sklearn import utils
import numpy as np
from keras import optimizers
from keras.models import load_model
from sklearn.metrics import accuracy_score, classification_report
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, GRU, Bidirectional

C:\Users\User\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\User\Anaconda3\lib\site-packages\pattern-2.6-py3.6.egg\pattern\text\en\..\..\..\..\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using 

In [2]:
# train_set = pd.read_csv('./corpus/tripadvisor/train_set.csv')
test_set = pd.read_csv('./corpus/tripadvisor/test.csv')
test_set.head()

,content,polarity
0,lokasi hotel tidak jauh dari komplek mall kali...,negative
1,lokasi hotel tidak jauh dengan taksi ke area m...,negative
2,lokasi hotel yang sangat strategis masih di ko...,negative
3,lokasi hotel yang sangat strategis masih di ko...,negative
4,lokasi hotel yang strategis di tengah kota dan...,negative


In [3]:
def sentiment_label(polarity):
    if polarity=='negative':
        return 0
    else:
        return 1

In [4]:
# train_set['sentiment'] = train_set['polarity'].apply(sentiment_label)
test_set['sentiment'] = test_set['polarity'].apply(sentiment_label)
test_set.head()

,content,polarity,sentiment
0,lokasi hotel tidak jauh dari komplek mall kali...,negative,0
1,lokasi hotel tidak jauh dengan taksi ke area m...,negative,0
2,lokasi hotel yang sangat strategis masih di ko...,negative,0
3,lokasi hotel yang sangat strategis masih di ko...,negative,0
4,lokasi hotel yang strategis di tengah kota dan...,negative,0


In [5]:
SEED = 2000

x_train, x_validation, y_train, y_validation = train_test_split(test_set['content'], test_set['sentiment'], test_size=.1, random_state=SEED)

In [6]:
# x_train = train_set['content']
# x_validation = test_set['content']
# y_train = train_set['sentiment']
# y_validation = test_set['sentiment']

In [6]:
def labelize_text(text,label):
    result = []
    prefix = label
    for i, t in zip(text.index, text):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result
  
x_train = labelize_text(x_train, 'TRAIN')
x_validation = labelize_text(x_validation, 'TEST')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [7]:
MAX_SEQUENCE_LENGTH = 85
data_dim = 500

In [8]:
word2vec = Word2Vec.load('./prosa-w2v/prosa.vec')
# word2vec = Word2Vec.load("./vectorizer/tripadvisor/word2vec_300.model")

In [9]:
def build_Word_Vector(tokens, size):
    vec = np.zeros((MAX_SEQUENCE_LENGTH - len(tokens), size))
    for word in tokens:
        try:
            vec = np.append(vec, word2vec[word])
        except KeyError: 
            vec = np.append(vec, np.zeros((1, size)))
            continue
    vec.reshape(MAX_SEQUENCE_LENGTH, size)
    return vec

In [10]:
train_vecs = np.concatenate([[build_Word_Vector(z, 500)] for z in tqdm(map(lambda x: x.words, x_train))])
val_vecs = np.concatenate([[build_Word_Vector(z, 500)] for z in tqdm(map(lambda x: x.words, x_validation))])

0it [00:00, ?it/s]C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
746it [00:01, 432.52it/s]
83it [00:00, 413.39it/s]


In [11]:
batch_size = 56
num_epochs = 10
hidden_size = 10
timesteps = MAX_SEQUENCE_LENGTH
num_class = 1

In [12]:
num_data = len(train_vecs)
num_data_val = len(val_vecs)

train_vecs = train_vecs.reshape((num_data, timesteps, data_dim))
y_train = y_train.reshape((num_data, num_class))
val_vecs = val_vecs.reshape((num_data_val, timesteps, data_dim))
y_validation = y_validation.reshape((num_data_val, num_class))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys


In [13]:
model = Sequential()
model.add(Bidirectional(GRU(hidden_size, input_shape=(timesteps, data_dim)), merge_mode='concat'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_vecs, y_train, epochs=num_epochs, validation_data=[val_vecs, y_validation])

Train on 746 samples, validate on 83 samples
Epoch 1/10
746/746 [==============================] - ETA: 1:40 - loss: 0.8175 - acc: 0.468 - ETA: 50s - loss: 0.7527 - acc: 0.531 - ETA: 33s - loss: 0.7875 - acc: 0.45 - ETA: 24s - loss: 0.7879 - acc: 0.46 - ETA: 19s - loss: 0.7925 - acc: 0.46 - ETA: 15s - loss: 0.7880 - acc: 0.46 - ETA: 13s - loss: 0.7756 - acc: 0.47 - ETA: 11s - loss: 0.7864 - acc: 0.47 - ETA: 9s - loss: 0.7798 - acc: 0.4792 - ETA: 8s - loss: 0.7723 - acc: 0.481 - ETA: 7s - loss: 0.7669 - acc: 0.480 - ETA: 6s - loss: 0.7594 - acc: 0.487 - ETA: 5s - loss: 0.7581 - acc: 0.485 - ETA: 4s - loss: 0.7527 - acc: 0.488 - ETA: 3s - loss: 0.7448 - acc: 0.495 - ETA: 3s - loss: 0.7378 - acc: 0.502 - ETA: 2s - loss: 0.7397 - acc: 0.501 - ETA: 2s - loss: 0.7438 - acc: 0.503 - ETA: 1s - loss: 0.7367 - acc: 0.514 - ETA: 1s - loss: 0.7314 - acc: 0.517 - ETA: 0s - loss: 0.7271 - acc: 0.520 - ETA: 0s - loss: 0.7297 - acc: 0.521 - ETA: 0s - loss: 0.7289 - acc: 0.521 - 9s 12ms/step - loss: 0.

746/746 [==============================] - ETA: 3s - loss: 0.4058 - acc: 0.843 - ETA: 3s - loss: 0.4023 - acc: 0.859 - ETA: 3s - loss: 0.3589 - acc: 0.885 - ETA: 3s - loss: 0.3744 - acc: 0.867 - ETA: 2s - loss: 0.3758 - acc: 0.862 - ETA: 2s - loss: 0.3795 - acc: 0.864 - ETA: 2s - loss: 0.3731 - acc: 0.870 - ETA: 2s - loss: 0.3696 - acc: 0.875 - ETA: 2s - loss: 0.3683 - acc: 0.871 - ETA: 2s - loss: 0.3786 - acc: 0.859 - ETA: 1s - loss: 0.3695 - acc: 0.866 - ETA: 1s - loss: 0.3653 - acc: 0.869 - ETA: 1s - loss: 0.3660 - acc: 0.870 - ETA: 1s - loss: 0.3660 - acc: 0.866 - ETA: 1s - loss: 0.3706 - acc: 0.862 - ETA: 1s - loss: 0.3825 - acc: 0.849 - ETA: 1s - loss: 0.3789 - acc: 0.854 - ETA: 0s - loss: 0.3776 - acc: 0.855 - ETA: 0s - loss: 0.3746 - acc: 0.860 - ETA: 0s - loss: 0.3720 - acc: 0.860 - ETA: 0s - loss: 0.3700 - acc: 0.863 - ETA: 0s - loss: 0.3687 - acc: 0.862 - ETA: 0s - loss: 0.3654 - acc: 0.864 - 4s 5ms/step - loss: 0.3648 - acc: 0.8646 - val_loss: 0.4756 - val_acc: 0.7831
Epoch

In [ ]:
# model.save('./model/bi_gru/bi_gru_model_01.h5')  

In [14]:
# model = load_model('./model/bi_gru/bi_gru_model_01.h5')
y_pred = model.predict(val_vecs)
for i in range(len(y_pred)):
    y_pred[i][0] = round(y_pred[i][0])

print("Accuracy: ", accuracy_score(y_validation, y_pred))
print(classification_report(y_validation, y_pred, labels = [0, 1], digits=8))

Accuracy:  0.8192771084337349
             precision    recall  f1-score   support

          0  0.91666667 0.80000000 0.85436893        55
          1  0.68571429 0.85714286 0.76190476        28

avg / total  0.83875502 0.81927711 0.82317620        83

